In [5]:
import cv2
import time
import os
import winsound
import pickle
import numpy as np
# import datetime
# #load test iamge
# test1 = cv2.imread(img_name)

# #convert the test image to gray image as opencv face detector expects gray images 
# gray_img = cv2.cvtColor(test1, cv2.COLOR_BGR2GRAY)

# cv2.imshow('Test Imag', gray_img) 
# cv2.waitKey(0) 
# cv2.destroyAllWindows()


In [6]:
person_name=input('target name: ')
flag = ' ' in person_name or '.' in person_name or '-' in person_name or '/' in person_name
while person_name=='' or flag == True:
    person_name=input('target name(use alphabet and "_" only): ')
    flag = ' ' in person_name or '.' in person_name or '-' in person_name or '/' in person_name


# if os.path.isdir(person_name)==True:
#     img_path=person_name
# else:
#     os.mkdir(person_name)
#     img_path=person_name
    
    

target name: masha


In [7]:
import numpy as np
import cv2
# import pickle

cap = cv2.VideoCapture(0)
# cap2 = cv2.VideoCapture(1)

ROI = []
idx=0

print('press "p" to take a picture.')
print('press "q" to quit.')
print('press "s" to save.')

while(True):

    ret, frame = cap.read()

    cv2.imshow('frame',frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):    
        break
    elif cv2.waitKey(1) & 0xFF == ord('p'):
        ROI.append(frame)
        idx += 1
        print('picture: '+str(idx))
    elif cv2.waitKey(1) & 0xFF == ord('s'):
        idx = 0
        if os.path.exists('image/'+person_name):
            
        else:
            os.mkdir('image/'+person_name)
        for img in ROI:
            idx+=1
            cv2.imwrite('image/'+person_name+'/'+str(idx)+'.jpg',img)
        print('done saving.')
        break
cap.release()
# cap2.release()
cv2.destroyAllWindows()


press "p" to take a picture.
press "q" to quit.
press "s" to save.
picture: 1
picture: 2
picture: 3
picture: 4
picture: 5
picture: 6
picture: 7
picture: 8
picture: 9
picture: 10
picture: 11
picture: 12
picture: 13
picture: 14
picture: 15
picture: 16
picture: 17
picture: 18
picture: 19
picture: 20
picture: 21
picture: 22
picture: 23
picture: 24
picture: 25
picture: 26
picture: 27
picture: 28
picture: 29
picture: 30
picture: 31
picture: 32
picture: 33
picture: 34
picture: 35
picture: 36
picture: 37
picture: 38
picture: 39
picture: 40
picture: 41
picture: 42
picture: 43
picture: 44
picture: 45
picture: 46
picture: 47
picture: 48
picture: 49
picture: 50
picture: 51
picture: 52
picture: 53
picture: 54
picture: 55
picture: 56
picture: 57
picture: 58
picture: 59
picture: 60
picture: 61
picture: 62
picture: 63
picture: 64
picture: 65
picture: 66
picture: 67
picture: 68
2
done saving.


In [4]:
import facenet
import tensorflow as tf
import math


# 專案的根目錄路徑
ROOT_DIR = os.getcwd()

# 模型的資料目錄
MODEL_PATH = os.path.join(ROOT_DIR, "model")

# MTCNN的模型
MTCNN_MODEL_PATH = os.path.join(MODEL_PATH, "mtcnn")

# MTCNN的模型
FACENET_MODEL_PATH = os.path.join(MODEL_PATH, "facenet/20170512-110547/20170512-110547.pb")

# linearSVM
SVM_MODEL_PATH = os.path.join(MODEL_PATH, "linearsvm")


# 訓練/驗證用的圖像資料目錄
IMG_IN_PATH = os.path.join(ROOT_DIR, "image")

# 訓練/驗證用的圖像資料目錄
IMG_OUT_PATH = os.path.join(ROOT_DIR, "image_crops")

IMG_TEMP_PATH = os.path.join(ROOT_DIR, "image_crops_temp")





C:\Users\shuoh\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
# 載入Facenet模型
print('Loading feature extraction model')
facenet.load_model(FACENET_MODEL_PATH)

# 取得模型的輸入與輸出的佔位符
images_placeholder = tf.get_default_graph().get_tensor_by_name("input:0")
embeddings = tf.get_default_graph().get_tensor_by_name("embeddings:0")
phase_train_placeholder = tf.get_default_graph().get_tensor_by_name("phase_train:0")
embedding_size = embeddings.get_shape()[1]

# 打印"人臉特徵向量"的向量大小
print("Face embedding size: ", embedding_size)

minsize = 40  # 最小的臉部的大小
threshold = [0.6, 0.7, 0.7]  # 三個網絡(P-Net, R-Net, O-Net)的閥值
factor = 0.709  # scale factor

margin = 44 # 在裁剪人臉時的邊框margin
image_size = 182 # 160 + 22

batch_size = 1000
input_image_size = 160

# 創建Tensorflow Graph物件
tf_g = tf.Graph().as_default()
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.6)

# 創建Tensorflow Session物件
tf_sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options, log_device_placement=False))

# 把這個Session設成預設的session
tf_sess.as_default()

import detect_face
# 載入MTCNN模型 (偵測人臉位置)
pnet, rnet, onet = detect_face.create_mtcnn(tf_sess, MTCNN_MODEL_PATH)

Loading feature extraction model
Model filename: D:\程式語言\anaconda\opencv\code_from_pi\model\facenet/20170512-110547/20170512-110547.pb
Face embedding size:  128
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [6]:
# 臉類別(ImageClass)的列表與圖像路徑
dataset = facenet.get_dataset(IMG_IN_PATH)
print('Creating networks and loading parameters')
with tf.Graph().as_default():
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.5)
    sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options, log_device_placement=False))
    with sess.as_default():
        pnet, rnet, onet = detect_face.create_mtcnn(sess, MTCNN_MODEL_PATH)
minsize = 20  # 最小的臉部的大小
threshold = [0.6, 0.7, 0.7]  # 三個網絡(P-Net, R-Net, O-Net)的閥值
factor = 0.709  # scale factor

margin = 44 # 在裁剪人臉時的邊框margin
image_size = 182 # 160 + 22
# 將一個隨機key添加到圖像檔名以允許使用多個進程進行人臉對齊
random_key = np.random.randint(0, high=99999)
bounding_boxes_filename = os.path.join(IMG_OUT_PATH, 'bounding_boxes_%05d.txt' % random_key)

Creating networks and loading parameters


In [7]:
from scipy import misc
from tqdm import tqdm
# 使用Tensorflow來運行MTCNN
with open(bounding_boxes_filename, "w") as text_file:
    nrof_images_total = 0 # 處理過的圖像總數
    nrof_successfully_aligned = 0 # 人臉圖像align的總數
    
    # 迭代每一個人臉身份(ImageClass)
    for cls in dataset:
        output_class_dir = os.path.join(IMG_OUT_PATH, cls.name) # 裁剪後的圖像目錄
        output_class_dir_temp = os.path.join(IMG_TEMP_PATH, cls.name) # 裁剪後的圖像目錄

        if not os.path.exists(output_class_dir):
            os.makedirs(output_class_dir)
            os.makedirs(output_class_dir_temp)

        
        # 迭代每一個人臉身份的圖像的路徑 (ImageClass.image_paths)
        for image_path in cls.image_paths:
            nrof_images_total += 1
            filename = os.path.splitext(os.path.split(image_path)[1])[0] # 取得圖像檔名
            output_filename = os.path.join(output_class_dir, filename + '.png') # 設定輸出的圖像檔名
            output_filename_temp = os.path.join(output_class_dir_temp, filename + '.png')
            # print(image_path)
            
            if not os.path.exists(output_filename):
                try:
                    img = misc.imread(image_path) # 讀進圖檔
                    # print('read data dimension: ', img.ndim)                    
                except (IOError, ValueError, IndexError) as e:
                    errorMessage = '{}: {}'.format(image_path, e)
                    # print(errorMessage)
                else:
                    # 將圖檔轉換成numpy array (height, widith, color_channels)
                    if img.ndim < 2:
                        print('Unable to align "%s"' % image_path)
                        text_file.write('%s\n' % (output_filename))
                        text_file.write('%s\n' % (output_filename_temp))
                        continue
                    if img.ndim == 2:
                        img = facenet.to_rgb(img)
                        print('to_rgb data dimension: ', img.ndim)
                    img = img[:, :, 0:3]
                    # print('after data dimension: ', img.ndim)
                    
                    # 使用MTCNN來偵測人臉在圖像中的位置
                    bounding_boxes, _ = detect_face.detect_face(img, minsize, pnet, rnet, onet, threshold, factor)
                    nrof_faces = bounding_boxes.shape[0] # 偵測到的人臉總數
                    # print('detected_face: %d' % nrof_faces)
                    if nrof_faces > 0:
                        # 當有偵測到多個人臉的時候, 我們希望從中找到主要置中位置的人臉
                        det = bounding_boxes[:, 0:4]
                        img_size = np.asarray(img.shape)[0:2]
                        if nrof_faces > 1:
                            bounding_box_size = (det[:, 2] - det[:, 0]) * (det[:, 3] - det[:, 1])
                            img_center = img_size / 2
                            offsets = np.vstack([(det[:, 0] + det[:, 2]) / 2 - img_center[1],
                                                 (det[:, 1] + det[:, 3]) / 2 - img_center[0]])
                            offset_dist_squared = np.sum(np.power(offsets, 2.0), 0)
                            index = np.argmax(bounding_box_size - offset_dist_squared * 2.0)  # some extra weight on the centering
                            det = det[index, :]
                        det = np.squeeze(det)
                        bb_temp = np.zeros(4, dtype=np.int32)
                        # 取得人臉的左上角與右下角座標
                        bb_temp[0] = det[0]
                        bb_temp[1] = det[1]
                        bb_temp[2] = det[2]
                        bb_temp[3] = det[3]
                        
                        # 進行裁剪以及大小的轉換
                        cropped_temp = img[bb_temp[1]:bb_temp[3], bb_temp[0]:bb_temp[2], :]
                        scaled_temp = misc.imresize(cropped_temp, (image_size, image_size), interp='bilinear')

                        nrof_successfully_aligned += 1
                        misc.imsave(output_filename, scaled_temp) # 儲存處理過的圖像
                        misc.imsave(output_filename_temp, scaled_temp)
                        text_file.write('%s %d %d %d %d\n' % (output_filename, bb_temp[0], bb_temp[1], bb_temp[2], bb_temp[3]))
                        text_file.write('%s %d %d %d %d\n' % (output_filename_temp, bb_temp[0], bb_temp[1], bb_temp[2], bb_temp[3]))
                    else:
                        # print('Unable to align "%s"' % image_path)
                        text_file.write('%s\n' % (output_filename))
                        text_file.write('%s\n' % (output_filename_temp))

print('Total number of images: %d' % nrof_images_total)
print('Number of successfully aligned images: %d' % nrof_successfully_aligned)

C:\Users\shuoh\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:28: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
C:\Users\shuoh\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:72: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
C:\Users\shuoh\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:75: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
C:\Users\shuoh\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:76: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


Total number of images: 188
Number of successfully aligned images: 73


In [8]:
# 使用Tensorflow的Facenet模型
# winsound.MessageBeep()
with tf.Graph().as_default():
    with tf.Session() as sess:
        datadir = IMG_TEMP_PATH # 經過偵測、對齊 & 裁剪後的人臉圖像目錄
        # 取得人臉類別(ImageClass)的列表與圖像路徑
        dataset = facenet.get_dataset(datadir)        
        # 原始: 取得每個人臉圖像的路徑與標籤
        paths, labels, labels_dict = facenet.get_image_paths_and_labels(dataset)        
        print('Origin: Number of classes: %d' % len(labels_dict))
        print('Origin: Number of images: %d' % len(paths))
        
        # 由於lfw的人臉圖像集中有很多的人臉類別只有1張的圖像, 對於訓練來說樣本太少
        # 因此我們只挑選圖像樣本張數大於5張的人臉類別
        
        # 過濾: 取得每個人臉圖像的路徑與標籤 (>=5)
        # facenet.get_image_paths_and_labels → 幫你把資料集中每筆資料的路徑與標籤回傳到paths, labels
        #                                      非常好用的函式!!!
        paths, labels, labels_dict = facenet.get_image_paths_and_labels(dataset, enable_filter=True, filter_size=2)        
        print('Filtered: Number of classes: %d' % len(labels_dict))
        print('Filtered: Number of images: %d' % len(paths))
            
        # 載入Facenet模型
        print('Loading feature extraction model')
        facenet.load_model(FACENET_MODEL_PATH)
        
        # 將facenet'計算圖(graph)'中需要的'節點(node)'取出來使用
        #           取得'input:0'放input，之後餵feed_dict進去
        #           取得'embeddings:0'取回output，再將特徵值存進私人資料庫
        #           取得'phase_train:0'加入條件至feed_dict
        images_placeholder = tf.get_default_graph().get_tensor_by_name("input:0")
        embeddings = tf.get_default_graph().get_tensor_by_name("embeddings:0")
        phase_train_placeholder = tf.get_default_graph().get_tensor_by_name("phase_train:0")
        embedding_size = embeddings.get_shape()[1]
        # 打印"人臉特徵向量"的向量大小
        print("Face embedding size: ", embedding_size)
        
        # 計算人臉特徵向量 (128 bytes)
        print('Calculating features for images')
        batch_size = 100 # 批次量
        image_size = 160  # 要做為Facenet的圖像輸入的大小
        
        nrof_images = len(paths) # 總共要處理的人臉圖像
        # 計算總共要跑的批次數
        nrof_batches_per_epoch = int(math.ceil(1.0 * nrof_images / batch_size))
        # 構建一個變數來保存"人臉特徵向量"
        emb_array = np.zeros((nrof_images, embedding_size)) # <-- Face Embedding
        
        for i in range(nrof_batches_per_epoch):
            start_index = i * batch_size
            end_index = min((i + 1) * batch_size, nrof_images)
            paths_batch = paths[start_index:end_index]
            images = facenet.load_data(paths_batch, False, False, image_size)
            feed_dict = {images_placeholder: images, phase_train_placeholder: False}
            emb_array[start_index:end_index, :] = sess.run(embeddings, feed_dict=feed_dict)
winsound.MessageBeep()

Origin: Number of classes: 11
Origin: Number of images: 188
Filtered: Number of classes: 2
Filtered: Number of images: 188
Loading feature extraction model
Model filename: D:\程式語言\anaconda\opencv\code_from_pi\model\facenet/20170512-110547/20170512-110547.pb
Face embedding size:  128
Calculating features for images


In [9]:
print(type(emb_array))
print(labels)
print(paths)
print(labels_dict)

<class 'numpy.ndarray'>
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
['D:\\程式語言\\anaconda\\opencv\\code_from_pi\\image_crops_temp\\henry\\1.png', 'D:\\程式語言\\anaconda\\opencv\\code_from_pi\\image_crops_temp\\henry\\10.png', 'D:\\程式語言\\anaconda\\opencv\\code_from_pi\\image_crops_temp\\henry\\11.png', 'D:\\程式語言\\anaconda\\opencv\\code_from_pi\\image_crops_temp\\henry\\12.png', 'D:\\程式語言\\anaconda\\opencv\\code_from_pi\\image_crops_temp\\henry\\13.png', 'D:\\程式語言\\anaconda\\openc

In [10]:

with open(os.path.join(IMG_IN_PATH,'lfw_emb_features.pkl'), 'rb') as emb_features_file:
    original_emb_features =pickle.load(emb_features_file)

# "人臉embedding"所對應的標籤(label)的資料
with open(os.path.join(IMG_IN_PATH,'lfw_emb_labels.pkl'), 'rb') as emb_lables_file:
    original_emb_labels =pickle.load(emb_lables_file)

# "標籤(label)對應到人臉名稱的字典的資料
with open(os.path.join(IMG_IN_PATH,'lfw_emb_labels_dict.pkl'), 'rb') as emb_lables_dict_file:
    original_emb_labels_dict =pickle.load(emb_lables_dict_file)

In [11]:
labels_new = np.array(labels)+len(original_emb_labels_dict)   # 新的labels總類不能從零開始，要將舊的labels總類考慮進去
labels_new = list(labels_new)
print(labels_new)
print(original_emb_labels[-1])
original_emb_labels.extend(labels_new)
print(original_emb_labels[-1])

print(type(original_emb_features))
print('______________')
print(type(emb_array))
new_emb_features = np.concatenate((original_emb_features,emb_array),axis=0)   #axis應該是0，待確認!!!!!!
print(new_emb_features.shape)



[1798, 1798, 1798, 1798, 1798, 1798, 1798, 1798, 1798, 1798, 1798, 1798, 1798, 1798, 1798, 1798, 1798, 1798, 1798, 1798, 1798, 1798, 1798, 1798, 1798, 1798, 1798, 1798, 1798, 1798, 1798, 1798, 1798, 1798, 1798, 1798, 1798, 1798, 1798, 1798, 1798, 1798, 1798, 1798, 1798, 1798, 1798, 1798, 1798, 1798, 1798, 1798, 1798, 1798, 1798, 1798, 1798, 1798, 1798, 1798, 1798, 1798, 1798, 1798, 1798, 1798, 1798, 1798, 1798, 1798, 1798, 1798, 1798, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 179

In [19]:
print(labels_dict)
b = len(original_emb_labels_dict)
for k, v in labels_dict.items():
    a =int(k)
    print(a)
    original_emb_labels_dict[a+b]=v

print(original_emb_labels_dict)

{0: 'henry', 1: 'huamg'}
0
1
{0: 'Aaron_Peirsol', 1: 'Aaron_Sorkin', 2: 'Abdel_Nasser_Assidi', 3: 'Abdoulaye_Wade', 4: 'Abdullah', 5: 'Abdullah_Gul', 6: 'Abdullah_al-Attiyah', 7: 'Abdullatif_Sener', 8: 'Abel_Pacheco', 9: 'Abid_Hamid_Mahmud_Al-Tikriti', 10: 'Adam_Sandler', 11: 'Adam_Scott', 12: 'Adel_Al-Jubeir', 13: 'Adolfo_Aguilar_Zinser', 14: 'Adolfo_Rodriguez_Saa', 15: 'Adrian_McPherson', 16: 'Adrian_Nastase', 17: 'Adrien_Brody', 18: 'Ahmad_Masood', 19: 'Ahmed_Chalabi', 20: 'Ahmet_Necdet_Sezer', 21: 'Ai_Sugiyama', 22: 'Aicha_El_Ouafi', 23: 'Aitor_Gonzalez', 24: 'Akbar_Hashemi_Rafsanjani', 25: 'Akhmed_Zakayev', 26: 'Al_Davis', 27: 'Al_Gore', 28: 'Al_Pacino', 29: 'Al_Sharpton', 30: 'Alan_Ball', 31: 'Alan_Greenspan', 32: 'Alan_Mulally', 33: 'Alastair_Campbell', 34: 'Albert_Costa', 35: 'Alberto_Fujimori', 36: 'Alberto_Ruiz_Gallardon', 37: 'Albrecht_Mentz', 38: 'Aldo_Paredes', 39: 'Alec_Baldwin', 40: 'Alejandro_Atchugarry', 41: 'Alejandro_Avila', 42: 'Alejandro_Toledo', 43: 'Aleksander_Kw

In [20]:
emb_features_file = open(os.path.join(IMG_IN_PATH,'lfw_emb_features_'+person_name+'.pkl'), 'wb')
pickle.dump(new_emb_features, emb_features_file)
emb_features_file.close()

# 保存"人臉embedding"所對應的標籤(label)的資料
emb_lables_file = open(os.path.join(IMG_IN_PATH,'lfw_emb_labels_'+person_name+'.pkl'), 'wb')      
pickle.dump(original_emb_labels, emb_lables_file)
emb_lables_file.close()

# 保存"標籤(label)對應到人臉名稱的字典的資料
emb_lables_dict_file = open(os.path.join(IMG_IN_PATH,'lfw_emb_labels_dict_'+person_name+'.pkl'), 'wb')
pickle.dump(original_emb_labels_dict, emb_lables_dict_file)
emb_lables_dict_file.close()

In [21]:
with open(os.path.join(IMG_IN_PATH,'lfw_emb_features_'+person_name+'.pkl'), 'rb') as emb_features_file:
    emb_features =pickle.load(emb_features_file)

# "人臉embedding"所對應的標籤(label)的資料
with open(os.path.join(IMG_IN_PATH,'lfw_emb_labels_'+person_name+'.pkl'), 'rb') as emb_lables_file:
    emb_labels =pickle.load(emb_lables_file)

# "標籤(label)對應到人臉名稱的字典的資料
with open(os.path.join(IMG_IN_PATH,'lfw_emb_labels_dict_'+person_name+'.pkl'), 'rb') as emb_lables_dict_file:
    emb_labels_dict =pickle.load(emb_lables_dict_file)

In [22]:
print(emb_features.shape)
print(len(emb_labels))
print(emb_labels_dict)

(12668, 128)
12668
{0: 'Aaron_Peirsol', 1: 'Aaron_Sorkin', 2: 'Abdel_Nasser_Assidi', 3: 'Abdoulaye_Wade', 4: 'Abdullah', 5: 'Abdullah_Gul', 6: 'Abdullah_al-Attiyah', 7: 'Abdullatif_Sener', 8: 'Abel_Pacheco', 9: 'Abid_Hamid_Mahmud_Al-Tikriti', 10: 'Adam_Sandler', 11: 'Adam_Scott', 12: 'Adel_Al-Jubeir', 13: 'Adolfo_Aguilar_Zinser', 14: 'Adolfo_Rodriguez_Saa', 15: 'Adrian_McPherson', 16: 'Adrian_Nastase', 17: 'Adrien_Brody', 18: 'Ahmad_Masood', 19: 'Ahmed_Chalabi', 20: 'Ahmet_Necdet_Sezer', 21: 'Ai_Sugiyama', 22: 'Aicha_El_Ouafi', 23: 'Aitor_Gonzalez', 24: 'Akbar_Hashemi_Rafsanjani', 25: 'Akhmed_Zakayev', 26: 'Al_Davis', 27: 'Al_Gore', 28: 'Al_Pacino', 29: 'Al_Sharpton', 30: 'Alan_Ball', 31: 'Alan_Greenspan', 32: 'Alan_Mulally', 33: 'Alastair_Campbell', 34: 'Albert_Costa', 35: 'Alberto_Fujimori', 36: 'Alberto_Ruiz_Gallardon', 37: 'Albrecht_Mentz', 38: 'Aldo_Paredes', 39: 'Alec_Baldwin', 40: 'Alejandro_Atchugarry', 41: 'Alejandro_Avila', 42: 'Alejandro_Toledo', 43: 'Aleksander_Kwasniewski'

In [23]:
# 準備相關變數
X_train = []; y_train = []
X_test = []; y_test = []

# 保存己經有處理過的人臉label
processed = set()

# 分割訓練資料集與驗證資料集
for (emb_feature, emb_label) in zip(emb_features, emb_labels):
    if emb_label in processed:
        X_train.append(emb_feature)
        y_train.append(emb_label)
    else:
        X_test.append(emb_feature)
        y_test.append(emb_label)
        processed.add(emb_label)

# 結果
print('X_train: {}, y_train: {}'.format(len(X_train), len(y_train)))
print('X_test: {}, y_test: {}'.format(len(X_test), len(y_test)))

X_train: 10868, y_train: 10868
X_test: 1800, y_test: 1800


In [27]:
from sklearn.svm import LinearSVC
# import winsound
# 訓練分類器
print('Training classifier')
linearsvc_classifier = LinearSVC(C=10, multi_class='ovr')

# 進行訓練
linearsvc_classifier.fit(X_train, y_train)

# 使用驗證資料集來檢查準確率
score = linearsvc_classifier.score(X_test, y_test)

# 打印分類器的準確率
print("Validation result: ", score)
# winsound.MessageBeep()

Training classifier
Validation result:  0.8766666666666667


In [25]:
a = list(emb_labels_dict.keys())
a = list(map(int, a))
# print(a)
# print(emb_labels_dict.keys())
print(emb_labels_dict[1798])

henry


In [26]:
# 產生一個人臉的人名列表，以便辨識後來使用
#class_names = [cls.name.replace('_', ' ') for cls in dataset]
member_list = []
class_names = []
for key in sorted(a):
    print(key)
    if emb_labels_dict[key] in member_list:
        class_names.append('Unknown')
#         class_names.append(emb_labels_dict[key].replace('_', ' '))

    else:
#         class_names.append('Unknown')
        class_names.append(emb_labels_dict[key].replace('_', ' '))
print(class_names)

# 保存人臉分類器到檔案系統
with open(SVM_MODEL_PATH+'\lfw_svm_classifier_'+person_name+'.pkl', 'wb') as outfile:
    pickle.dump((linearsvc_classifier, class_names), outfile)  #class_names可用客製鍵值對給對應商家
    
print('Saved classifier model to file lfw_svm_classifier_'+person_name+'.pkl')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27